In [3]:
pwd

'/home/chang/domain_intent'

In [4]:
cd ..

/home/chang


In [5]:
cd domain_intent/SlotGated-SLU/data/snips/train

/home/chang/domain_intent/SlotGated-SLU/data/snips/train


In [7]:
ls -a

./  ../  .ipynb_checkpoints/  label  seq.in  seq.out


---data_generation 코드---

In [8]:
#only extract sentence from snips dataset
f = open('seq.in', 'r')
x_train = []
while True:
    line = f.readline()
    x_str = line[0:-1]
    x_train.append(x_str)
    if not line:
        break

f.close()

In [9]:
del x_train[-1]
print(x_train[0])

listen to westbam alumb allergic on google music


In [10]:
print(len(x_train))

13084


In [11]:
#only extract label from snips dataset
f = open('label','r')
y_train = []
while True:
  line = f.readline()
  y_str = line[0:-1]
  y_train.append(y_str)
  if not line: break

f.close()

del y_train[-1]

print(len(y_train))
print(y_train[0])

13084
PlayMusic


In [12]:
#only extract sentence labeled 'PlayMusic' or 'AddToPlaylist'
refined_x_train = []
refined_y_train = []
for i in range(len(y_train)):
    if y_train[i] == 'PlayMusic' or y_train[i] == 'AddToPlaylist':
        refined_x_train.append(x_train[i])
        refined_y_train.append('PlayMusicOrVideo')

print(len(refined_x_train))
print(len(refined_y_train))
print(refined_x_train[0])
print(refined_y_train[0])

3732
3732
listen to westbam alumb allergic on google music
PlayMusicOrVideo


In [13]:
#only extract sentence not labeled 'PlayMusic' or 'AddToPlaylist'
#match the ratio with sentence labeled 'PlayMusic' or 'AddToPlaylist'
cnt = 0

for i in range(len(y_train)):
    if y_train[i] != 'PlayMusic' and y_train[i] != 'AddToPlaylist':
        if cnt < 3733:
            refined_x_train.append(x_train[i])
            refined_y_train.append('NotPlayMusicOrVideo')
            cnt += 1

print(len(refined_x_train))
print(len(refined_y_train))
print(refined_x_train[0])
print(refined_y_train[0])

7465
7465
listen to westbam alumb allergic on google music
PlayMusicOrVideo


In [14]:
import nlpaug
import nlpaug.augmenter.word as naw
import nlpaug.flow as naf
import nltk
nltk.download('averaged_perceptron_tagger')

2022-07-22 10:15:27.831255: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
#generate test data using nlpaug
TOPK=20 #default=100
ACT = 'insert' #"substitute"
test_sentence = refined_x_train[4]


aug_bert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', 
    #device='cuda',
    action=ACT, top_k=TOPK)
print("Original:")
print(test_sentence)
print("Augmented Text:")
for i in range(5):
    augmented_text = aug_bert.augment(test_sentence)
    print(augmented_text)

Original:
add slimm cutta calhoun to my this is prince playlist  
Augmented Text:
add slimm deluxe cutta calhoun tape to my ipod this one is prince playlist
add slimm to cutta calhoun soundtrack to my this movie is prince claus playlist
add slimm cutta calhoun intro to dub my this morning is prince playlist
add up slimm cutta calhoun to see my this mixtape is my prince playlist
gonna add another slimm cutta calhoun to my music this is prince playlist


In [18]:
# model_type: word2vec, glove or fasttext
aug_w2v = naw.WordEmbsAug(
    model_type='glove', model_path='/home/chang/domain_intent/glove.6B.300d.txt',
    action="substitute")
print("Original:")
print(test_sentence)

Original:
add slimm cutta calhoun to my this is prince playlist  


In [19]:
#test case
text = refined_x_train[0]
print('orginal sentence:')
print(text)

aug = naf.Sequential([
    aug_bert,aug_w2v
])

aug.augment(text, n=10)

orginal sentence:
listen to westbam alumb allergic on google music


['persuade myself to google westbam alumb allergic tracks putting advertisers music',
 "“ explain to fugitive westbam alumb toxicity on 's google music",
 'attentively to mixtape westbam would alumb dermatitis files on google music',
 'listen to the westbam made alumb allergic available their websites got music',
 'asked to eric westbam present alumb autoimmune on google mobile singing',
 "n't behind seeking fatah westbam alumb allergic on google cd music",
 'everybody move to westbam saudi alumb allergic on your google basic pianist',
 'listen directly follow westbam as alumb allergic live along revver google music',
 'downloaded listen direct to westbam aka alumb allergic available on ibm performers',
 'though listen go westbam with alumb allergic available on computer play label']

In [20]:
import pandas as pd
csv_data = pd.read_csv('/home/chang/domain_bible_dataset/dataset.csv')

In [21]:
print(csv_data)

                                                   verse      label
0       listen to westbam alumb allergic on google music  not bible
1            add step to me to the 50 clásicos playlist   not bible
2      i give this current textbook a rating value of...  not bible
3                   play the song little robin redbreast  not bible
4      please add iris dement to my playlist this is ...  not bible
...                                                  ...        ...
26103  And because there was little food to be had in...      bible
26104  Now when he came near to Egypt, he said to Sar...      bible
26105  And I am certain that when the men of Egypt se...      bible
26106  And he said, I have had news that there is gra...      bible
26107  And the Lord came to him in a vision and said,...      bible

[26108 rows x 2 columns]


In [22]:
domain_df = csv_data[csv_data['label'] == 'bible']
print(domain_df)

                                                   verse  label
13084  Then all the people of Judah took Uzziah, who ...  bible
13085  Uzziah was sixteen years old when he became ki...  bible
13086  The Ammonites gave offerings to Uzziah: and ne...  bible
13087  Uzziah made towers in Jerusalem, at the doorwa...  bible
13088  In addition, Uzziah had an army of fighting-me...  bible
...                                                  ...    ...
26103  And because there was little food to be had in...  bible
26104  Now when he came near to Egypt, he said to Sar...  bible
26105  And I am certain that when the men of Egypt se...  bible
26106  And he said, I have had news that there is gra...  bible
26107  And the Lord came to him in a vision and said,...  bible

[13024 rows x 2 columns]


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

#generate sentence
TOPK=20 #default=100
ACT = 'insert' #"substitute"
test_sentence = refined_x_train[4]


aug_bert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', 
    #device='cuda',
    action=ACT, top_k=TOPK)

aug_w2v = naw.WordEmbsAug(
    model_type='glove', model_path='/home/chang/domain_intent/glove.6B.300d.txt',
    action="substitute")

aug = naf.Sequential([
    aug_bert,aug_w2v
])

for i in range(len(refined_x_train)):
    origin_sentence = refined_x_train[i]
    origin_data = {}
    origin_data['verse'] = origin_sentence
    origin_data['label'] = refined_y_train[i]
    domain_df = domain_df.append(origin_data, ignore_index=True)
    
    for j in range(3):
        augmented_text = aug.augment(origin_sentence, n=1)
        new_data = {}
        new_data['verse'] = augmented_text
        new_data['label'] = refined_y_train[i]
        domain_df = domain_df.append(new_data, ignore_index=True)
        

# print(domain_df)

In [56]:
print(domain_df)

                                                   verse                label
0      Then all the people of Judah took Uzziah, who ...                bible
1      Uzziah was sixteen years old when he became ki...                bible
2      The Ammonites gave offerings to Uzziah: and ne...                bible
3      Uzziah made towers in Jerusalem, at the doorwa...                bible
4      In addition, Uzziah had an army of fighting-me...                bible
...                                                  ...                  ...
42879  what very will the afghan good be like this we...  NotPlayMusicOrVideo
42880             rate this textbook two out of 6 points  NotPlayMusicOrVideo
42881  zero that textbook includes two point this sco...  NotPlayMusicOrVideo
42882  reduced wins this textbook followed tries out ...  NotPlayMusicOrVideo
42883  rate in this critique two pass out views only ...  NotPlayMusicOrVideo

[42884 rows x 2 columns]


In [57]:
domain_df.to_csv('/home/chang/domain_intent/domain_dataset.csv', mode='w',encoding='utf-8',index=False)

In [37]:
data_numpy = domain_df.to_numpy()

In [39]:
print(data_numpy)

[['Then all the people of Judah took Uzziah, who was sixteen years old, and made him king in place of his father Amaziah.'
  'bible']
 ["Uzziah was sixteen years old when he became king, and he was ruling in Jerusalem for fifty-two years his mother's name was Jechiliah of Jerusalem."
  'bible']
 ['The Ammonites gave offerings to Uzziah: and news of him went out as far as the limit of Egypt for he became very great in power.'
  'bible']
 ...
 ['rate this textbook two – out averages of 6 average points'
  'NotPlayMusicOrVideo']
 ['metacritic rate this article textbook two degrees out of 6 points'
  'NotPlayMusicOrVideo']
 ['rate this textbook score two stars out of its 6 points'
  'NotPlayMusicOrVideo']]


In [40]:
print(type(data_numpy))

<class 'numpy.ndarray'>


In [ ]:
# print(data_numpy[10][0])

In [58]:
print(refined_x_train[0])

listen to westbam alumb allergic on google music


In [77]:
from rake_nltk import Rake

test_sent = 'Play good music to listen to while sleeping'

r = Rake(min_length=1, max_length=4)
#r = Rake()
r.extract_keywords_from_text(test_sent)
r.get_ranked_phrases_with_scores()

[(9.0, 'play good music'), (1.0, 'sleeping'), (1.0, 'listen')]